In [ ]:
! pip install pytorch_pretrained_bert

In [ ]:
! pip install transformers

In [70]:
from transformers import BertForSequenceClassification, BertTokenizer
import torch
import numpy as np
import pandas as pd

In [63]:
finbert = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-tone',num_labels=3)
tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-tone')

In [24]:
def get_sentiment(tokens):
  output = model(**tokens)
  probabilities = torch.nn.functional.softmax(output[0],dim=-1)
  return probabilities

In [20]:
tokens = tokenizer.encode_plus(txt, add_special_tokens = False)
tokens.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [41]:
labels = {0:'neutral', 1:'positive',2:'negative'}

In [44]:
def get_data(path):
  texts = []
  labels = []
  with open(path, encoding= "ISO-8859-1") as f:
    lines = f.readlines()
  for line in lines:
    texts.append(line.split('@')[0])
    labels.append(line.split('@')[1])
  df_dict = {
      'text' : texts,
      'label' : labels
  }
  df = pd.DataFrame.from_dict(df_dict)
  return df

In [46]:
df = get_data('/content/drive/MyDrive/CMPE CISC 351 Project/FinancialPhraseBank/Sentences_50Agree.txt')

In [72]:
def get_articles(path):
  df = pd.read_csv(path)
  articles = []
  sentiments = []
  for i, row in df.iterrows():
    articles.append(row['Article Text'])
    if row['Sentiment Label'] == 'Positive':
      sentiments.append(1)
    elif row['Sentiment Label'] == 'Neutral':
      sentiments.append(0)
    else:
      sentiments.append(2)
  df_dict = {
      'text' : articles,
      'label' : sentiments
  }
  df = pd.DataFrame.from_dict(df_dict)
  return df

In [74]:
df_articles = get_articles('/content/drive/MyDrive/CMPE CISC 351 Project/Sentiment_Labelling.csv')

In [91]:
sent_val = list()
i = 0
true_count = 0
for count, row in df_articles.iterrows():
    txt = row['text']
    txt = txt[0:1500]
    true_label = row['label']
    if true_label == 1:
      true_label = 1
    elif true_label == 0.5:
      true_label = 0
    elif true_label == 0:
      true_label = 2
    true_label = labels[true_label]
    inputs = tokenizer(txt, return_tensors="pt", padding=True)
    outputs = finbert(**inputs)[0]

    if true_label == val:
      true_count += 1
    i += 1

    val = labels[np.argmax(outputs.detach().numpy())]
    print(true_label, '----', val)
    print(row['text'])
    print('#######################################################')    
    sent_val.append(val)

print("count of correctly classified: " + str(true_count))
print("total count: " + str(i))

positive ---- positive
PLANO, Texas, Dec. 8, 2020 /PRNewswire/ --European Wax Center(EWC), the leading personal care franchise brand that offers expert wax services from certified specialists is proud to welcome a new Chief Financial Officer, Jennifer Vanderveldt. In the midst of European Wax Center's accelerated growth plan, Jennifer will lead the Accounting and FP&A teams to continue to widen growth and organizational initiatives. (PRNewsfoto/European Wax Center) "We are thrilled to have Jennifer join the European Wax Center team and partner with all functions to provide analytical and insightful leadership to support our accelerated growth, operational efficiency, the 360-guest experience, as well as our people and performance strategies," said David Berg, CEO of European Wax Center. Jennifer's background includes 20 years of leadership experience across strategy, finance, operations, marketing, analytics, and insights with multibillion-dollar retail and CPG brands, including Rubio'